In [ ]:
import os
import pathlib
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
from xml.dom import minidom
import re
import json

from random import shuffle

In [ ]:
with open('workspace/road_damage/inference-config.json', 'r') as f:
    CONFIGURATION = json.load(f)


In [ ]:
'''
Doesnt redownload the file when already downloaded. Which is fantastic.
'''
def validate_and_load_existing_model():
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = CONFIGURATION['CHOSEN_MODEL'] + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=CONFIGURATION['CHOSEN_MODEL'], 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"
  print(model_dir)

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

MODEL = validate_and_load_existing_model()

In [ ]:
def load_from_path():
    print(CONFIGURATION['MODEL_PATH'])
    model = tf.saved_model.load(CONFIGURATION['MODEL_PATH'])
    model = model.signatures['serving_default']
    return model

MODEL = load_from_path()

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  category_index = label_map_util.create_category_index_from_labelmap(CONFIGURATION['PATH_TO_LABELS'], use_display_name=True)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))


In [ ]:
def find_best_inference():
    model = MODEL
    images_list = os.listdir(CONFIGURATION['IMAGES_PATH'])
    shuffle(images_list)
    for img_name in images_list:
        img_path = os.path.join(CONFIGURATION['IMAGES_PATH'], img_name)
        print('Try to find accurate enough inference for ', img_path)
        image_np = np.array(Image.open(img_path))
        try:
            output_dict = run_inference_for_single_image(model, image_np)
            if output_dict['detection_scores'][0] > 0.5:
                show_inference(MODEL, img_path)
                break
        except:
            print('Error occurred while processing',img_path)
        
find_best_inference()   

In [39]:
images_list = os.listdir(CONFIGURATION['IMAGES_PATH'])
shuffle(images_list)

image_absolute_name = os.path.join(CONFIGURATION['IMAGES_PATH'], images_list[0])
print(images_list[0], image_absolute_name)

show_inference(MODEL, image_absolute_name)



Try to find accurate enough inference for  D:/university/datasets/Potholes/images\320.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\616.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\340.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\345.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\552.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\85.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\107.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\412.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\533.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\66.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\610.jpg
Try to find 

Try to find accurate enough inference for  D:/university/datasets/Potholes/images\414.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\502.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\37.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\281.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\60.jpg
Error occurred while processing D:/university/datasets/Potholes/images\60.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\554.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\335.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\580.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\368.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\478.jpg
Error occurred while pro

Try to find accurate enough inference for  D:/university/datasets/Potholes/images\108.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\581.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\410.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\140.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\311.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\334.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\497.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\553.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\47.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\584.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\56.jpg
Try to find 

Try to find accurate enough inference for  D:/university/datasets/Potholes/images\590.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\241.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\57.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\351.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\367.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\36.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\441.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\329.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\175.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\246.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\149.jpg
Try to find 

Try to find accurate enough inference for  D:/university/datasets/Potholes/images\168.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\88.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\423.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\214.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\98.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\551.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\468.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\431.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\450.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\29.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\59.jpg
Try to find ac

Try to find accurate enough inference for  D:/university/datasets/Potholes/images\232.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\315.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\430.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\376.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\156.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\380.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\458.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\418.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\461.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\90.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\372.jpg
Try to find

Try to find accurate enough inference for  D:/university/datasets/Potholes/images\91.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\416.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\397.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\569.jpg
Error occurred while processing D:/university/datasets/Potholes/images\569.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\114.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\272.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\387.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\89.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\358.jpg
Try to find accurate enough inference for  D:/university/datasets/Potholes/images\25.jpg
Try to find accurate eno

In [ ]:
images_list = os.listdir(CONFIGURATION['IMAGES_PATH'])
shuffle(images_list)

image_absolute_name = os.path.join(CONFIGURATION['IMAGES_PATH'], images_list[0])
print(images_list[0], image_absolute_name)

show_inference(MODEL, image_absolute_name)

